In [1]:
import requests
import pandas as pd
import re

In [2]:
endpoint = "https://ocrkudosware.cognitiveservices.azure.com/"
key = "b186f7f03f6249b6870a6e0eb0ab5a8c"
pdf = open("test-2-scanned.pdf","rb")
pdf_data = pdf.read()

# Initial data
url = endpoint + "formrecognizer/v2.1/layout/analyze?language=en&pages=1"
header = {
    "Content-Type":"application/pdf",
    "Ocp-Apim-Subscription-Key":key
}
response = requests.post(url,headers=header,data=pdf_data)
requestId = response.headers["apim-request-id"]

In [6]:
result_url = "https://ocrkudosware.cognitiveservices.azure.com/formrecognizer/v2.1/layout/analyzeResults/" + requestId
results = requests.get(result_url,headers={"Ocp-Apim-Subscription-Key":key})

In [4]:
def normalize(summary_field):
    n = len(summary_field)
    count = 0
    rem = 0
    for i in range(n-1,0,-1):
        count += 1
        if(summary_field[i] == "."):
            rem = i
            break
    summary_field = summary_field.replace(".","#")
    summary_field = summary_field[:rem] + "." + summary_field[rem+1:]
    summary_field = summary_field.replace("#","")
    return summary_field
    
    
            
            
        
    
            
        

In [7]:
table_data = results.json()
initial_bal = table_data["analyzeResult"]["readResults"][0]["lines"]
Previous_Balance = 0
Payment_Credits = 0
Purchases = 0
Cash_Advances = 0
Balance_Transfers = 0
Fees_Charged = 0
Interest_Charged = 0
New_Balance = 0 

for i in range(len(initial_bal)):
    if(initial_bal[i]["text"] == "Previous Balance"):
        Previous_Balance = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "Payment, Credits"):
        Payment_Credits = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "Purchases"):
        Purchases = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "Cash Advances"):
        Cash_Advances = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "Balance Transfers"):
        Balance_Transfers = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "Fees Charged"):
        Fees_Charged = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "Interest Charged"):
        Interest_Charged = initial_bal[i+1]["text"]
    elif(initial_bal[i]["text"] == "New Balance"):
        New_Balance = initial_bal[i+1]["text"]
        
        
        
Previous_Balance = Previous_Balance.replace("$","")
Previous_Balance = float(Previous_Balance.replace(",",""))
print(Previous_Balance)

Payment_Credits = Payment_Credits.replace("$","")
Payment_Credits = float(Payment_Credits.replace(",",""))
print(Payment_Credits)

Purchases = Purchases.replace("$","")
Purchases = normalize(Purchases)
Purchases = float(Purchases.replace(",",""))
print(Purchases)

Cash_Advances = Cash_Advances.replace("$","")
Cash_Advances = float(Cash_Advances.replace(",",""))
print(Cash_Advances)

Balance_Transfers = Balance_Transfers.replace("$","")
Balance_Transfers = float(Balance_Transfers.replace(",",""))
print(Balance_Transfers)

Fees_Charged = Fees_Charged.replace("$","")
Fees_Charged = float(Fees_Charged.replace(",",""))
print(Fees_Charged)

Interest_Charged = Interest_Charged.replace("$","")
Interest_Charged = float(Interest_Charged.replace(",",""))
print(Interest_Charged)

New_Balance = New_Balance.replace("$","")
New_Balance = float(New_Balance.replace(",",""))

2052.42
-2052.42
3289.64
0.0
0.0
0.0
0.0


In [8]:
# validate amount 
val = Previous_Balance + Payment_Credits + Purchases + Cash_Advances + Balance_Transfers + Fees_Charged + Interest_Charged
val = "{:.2f}".format(val)
New_Balance = "{:.2f}".format(New_Balance)
print(val,New_Balance)
if(val == New_Balance):
    print("matched")
else:
    print("corrupted")

3289.64 3289.64
matched


In [9]:
# Transactions
url = endpoint + "formrecognizer/v2.1/layout/analyze?language=en&pages=3"
header = {
    "Content-Type":"application/pdf",
    "Ocp-Apim-Subscription-Key":key
}
response = requests.post(url,headers=header,data=pdf_data)
requestId = response.headers["apim-request-id"]



In [13]:
result_url = "https://ocrkudosware.cognitiveservices.azure.com/formrecognizer/v2.1/layout/analyzeResults/" + requestId
results = requests.get(result_url,headers={"Ocp-Apim-Subscription-Key":key})
results.status_code

200

In [14]:
table_data = results.json()


In [15]:
rows = table_data["analyzeResult"]["pageResults"][0]["tables"][0]["rows"]
cols = table_data["analyzeResult"]["pageResults"][0]["tables"][0]["columns"]
data = table_data["analyzeResult"]["pageResults"][0]["tables"][0]["cells"]

In [16]:
lst = [None] * rows
for i in range(len(data)):
    curr_row = data[i]["rowIndex"]
    curr_col = data[i]["columnIndex"]
    tpList = lst[curr_row]
    if(tpList == None):
        tpList = [None] * cols
    tpList[curr_col] = data[i]["text"]
    
    lst[curr_row] = tpList

In [17]:
df = pd.DataFrame(lst)
df.to_excel("check6.xlsx",index=False,header=False)

In [18]:
# verify transactions - 2
transactions = pd.read_excel("check6.xlsx")
transactions_amount = transactions[:]["$ Amount"]
transactions_date = transactions[:]["Date of Transaction"]
curr_total = 0
temp_total = 0
curr_type = ""
mapped_amount_type = {"PAYMENTS AND OTHER CREDITS": Payment_Credits,"PURCHASE":Purchases}
for i in range(len(transactions_amount)):
    if(type(1.0) == type(transactions_amount[i])):
        if(type(transactions_date[i]) == type(1.0)):
            print("pass")
        elif(transactions_date[i] == "PAYMENTS AND OTHER CREDITS"):
            current_type = mapped_amount_type["PAYMENTS AND OTHER CREDITS"]  
            
            temp_total = 0
        else:
            print(temp_total,current_type)
            if(current_type == temp_total):
                print("sub-matched")
            temp_total = 0
    else:
        transactions_amount[i] = transactions_amount[i].replace(",","")
        transactions_amount[i] = float(transactions_amount[i])
        transactions_amount[i] = "{:.2f}".format(transactions_amount[i])
        transactions_amount[i] = float(transactions_amount[i])
        curr_total += transactions_amount[i]
        curr_total = "{:.2f}".format(curr_total)
        curr_total = float(curr_total)
        
        temp_total += transactions_amount[i]
        temp_total = "{:.2f}".format(temp_total)
        temp_total = float(temp_total)
        
        
        

if(Purchases == temp_total):
    print("Last Matched ",Purchases, " = ",temp_total)

-2052.42 -2052.42
sub-matched
Last Matched  3289.64  =  3289.64


c:\users\darsh\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-18-7327b6e15135>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_amount[i] = float(transactions_amount[i])
<ipython-input-18-7327b6e15135>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_